In [49]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [50]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [51]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
  """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
  """
)


class Classification(BaseModel):
  sentiment: str = Field(description="The sentiment of the text")
  aggressiveness: int = Field(
    description="How aggressive the text is on a scale from 1 to 10"
  )
  language: str = Field(description="The language the text is written in")
  eng_ver: str = Field(description="The English version of the paragraph")
  
structured_llm = llm.with_structured_output(Classification)

In [52]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

In [53]:
response

Classification(sentiment='Positive', aggressiveness=1, language='Spanish', eng_ver="I'm incredibly happy to have met you! I think we'll be very good friends!")

In [54]:
response.model_dump()

{'sentiment': 'Positive',
 'aggressiveness': 1,
 'language': 'Spanish',
 'eng_ver': "I'm incredibly happy to have met you! I think we'll be very good friends!"}

## Finer control

In [58]:
class Classification(BaseModel):
  sentiment: str = Field(..., enumerate=["happy", "neutral", "sad"])
  aggressiveness: int = Field(..., description="describes how aggressive the statement is, the higher the number the more aggressive", enumerate=[1, 2, 3, 4, 5])
  language: str = Field(..., enumerate=["spanish", "english", "french", "german", "italian"])

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
  """
  Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
  """
)

# llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

In [60]:
test_llm = llm.with_structured_output(Classification)

In [65]:
input = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": input})
ans = test_llm.invoke(prompt)

In [66]:
ans

Classification(sentiment='Positive', aggressiveness=0, language='Spanish')